In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.").getOrCreate())

In [4]:
spark.sparkContext.setLogLevel("WARN")

In [5]:
spark?

Type:        SparkSession
String form: <pyspark.sql.session.SparkSession object at 0x00000242FC989FC0>
File:        c:\pyspark\spark-3.2.3-bin-hadoop3.2\python\pyspark\sql\session.py
Docstring:  
The entry point to programming Spark with the Dataset and DataFrame API.

A SparkSession can be used create :class:`DataFrame`, register :class:`DataFrame` as
tables, execute SQL over tables, cache tables, and read parquet files.
To create a :class:`SparkSession`, use the following builder pattern:

.. autoattribute:: builder
   :annotation:

Examples
--------
>>> spark = SparkSession.builder \
...     .master("local") \
...     .appName("Word Count") \
...     .config("spark.some.config.option", "some-value") \
...     .getOrCreate()

>>> from datetime import datetime
>>> from pyspark.sql import Row
>>> spark = SparkSession(sc)
>>> allTypes = sc.parallelize([Row(i=1, s="string", d=1.0, l=1,
...     b=True, list=[1, 2, 3], dict={"s": 0}, row=Row(a=1),
...     time=datetime(2014, 8, 1, 14, 1, 5

In [6]:
book = spark.read.text("./1342-0.txt")

In [7]:
# column_1 = book.select(book["value"])
# column_2 = book.select(book.value)

In [8]:
# column_1.show(10, truncate = 70)
# column_2.show(10, truncate = 70)

In [9]:
# book.show(15, truncate=100)

In [10]:
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

In [11]:
# import pyspark.sql.functions as F

# used functions:
#     1. col
#     2. explode
#     3. lower
#     4. split
#     5. regexp_extract

In [12]:
lines = book.select(split(col("value"), " ").alias("line"))

In [13]:
#lines = book.select(split(book.value, " ").alias("line"))

In [14]:
#lines = book.select(split(book.value, " ").alias("line"))

In [15]:
lines.show(10)

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
|[with, this, eBoo...|
|                  []|
|                  []|
|[Title:, Pride, a...|
|                  []|
+--------------------+
only showing top 10 rows



In [16]:
words = lines.select(explode(col("line")).alias("book"))

In [17]:
words.show(5)

+---------+
|     book|
+---------+
|      The|
|  Project|
|Gutenberg|
|    EBook|
|       of|
+---------+
only showing top 5 rows



In [18]:
# convert to lowercase

lower_words = words.select(lower(col("book")).alias("lower_word"))

In [19]:
lower_words.show(15)

+----------+
|lower_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
+----------+
only showing top 15 rows



In [20]:
#  remove characters and spaces

clean_words = lower_words.select(regexp_extract(col("lower_word"), "[a-z]+", 0).alias("clean_word"))

remove_spaces = clean_words.filter(col("clean_word") != "")

In [22]:
remove_spaces.show(15)

+----------+
|clean_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
| prejudice|
|        by|
|      jane|
|    austen|
|      this|
|     ebook|
|        is|
|       for|
+----------+
only showing top 15 rows



In [23]:
import pandas as pd

In [30]:
data = pd.read_csv(r"C:\Users\oluwa\Downloads\tmdb_movies.csv")

In [31]:
df = pd.DataFrame(data)

In [32]:
df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09
